## Definition and making initial generation

In [8]:
using DataFrames

┌ Info: Recompiling stale cache file /home/yoshinobu/.julia/compiled/v1.0/DataFrames/AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1187


In [ ]:
# Define proctable as a condition matrix for each material.
# Each material has its own process information.

# p1 : pipe cutting
# p2 : pre-bend-annealing
# p3 : icing (for material1 only)
# p4 : bending (rotation)
# p5 : bending (press) 
# p6 : pre-calib-anealing
# p7 : calibration (horizontal)
# p8 : calibration (vertical)
# p9 : destress-anealing
# p10 : buffing1 
# p11 : MC cutting
# p12 : buffing2

# mat1 is a horizontal bending 
# mat2 is a vertical bending

# informataion dictionary for process
proclist = ["pipe_cut","prebend-anneal","icing","rotation-bend","press-bend","precalib-anneal","h-calib","v-calib","destress","buff1","MC","buff2"]

In [202]:
# list of information for each material
mattable = [["m1" 50]; ["m2" 50]; ["m3" 100]]

3×2 Array{Any,2}:
 "m1"   50
 "m2"   50
 "m3"  100

In [191]:
proctable = [
    [1 3 0 1 0 3 2 0 0 2 1 0]; # horizontal
    [1 3 0 1 0 3 1 0 1 0 0 0]; # calib only 
    [1 0 0 0 0 3 0 2 0 0 1 2]; # vertical
]

3×12 Array{Int64,2}:
 1  3  0  1  0  3  2  0  0  2  1  0
 1  3  0  1  0  3  1  0  1  0  0  0
 1  0  0  0  0  3  0  2  0  0  1  2

In [192]:
size(proctable)

(3, 12)

In [32]:
function joblist(lst)
    a = []
    for i in 1:length(lst)
        append!(a,repeat([i],lst[i]))
    end
    return(a)
end

joblist (generic function with 1 method)

In [45]:
# make job time table base (ignoring sequence)
function jobtablebase(ptable)
    lp = sum(ptable)
    sz = size(ptable)
    
    tb = zeros(Int,sz[1],lp)
    for k in 1:sz[1]
        jl = joblist(ptable[k,:])
        tb[k,1:length(jl)] = jl
    end
    tb
end

jobtablebase (generic function with 1 method)

In [193]:
jtable = jobtablebase(proctable)

3×32 Array{Int64,2}:
 1  2  2  2  4  6   6   6   7  7  10  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  2  2  2  4  6   6   6   7  9   0     0  0  0  0  0  0  0  0  0  0  0  0
 1  6  6  6  8  8  11  12  12  0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [47]:
using Random

In [106]:
# sort list in order while keeping position of 0 entity
function possort!(jtbl)
    for i in 1:size(jtbl)[1]
        lst = jtbl[i,:]
        st = sort(lst[lst.>0]) # sorted value
        pos = range(1,stop=length(lst))[lst.>0] # index of non zero
    
        for k in 1:length(pos)
            lst[pos[k]] = st[k]
        end
        jtbl[i,:] = lst
    end
    jtbl
end

possort! (generic function with 1 method)

In [53]:
# make shuffled job table
function jobtable(ptable)
    jtable=jobtablebase(ptable)
    for i in 1:size(jtable)[1]
        jtable[i,:] = shuffle(jtable[i,:])
    end
    jtable
end

jobtable (generic function with 1 method)

In [194]:
jtable = jobtable(proctable)

3×32 Array{Int64,2}:
 0  7  0  2  0  0  0  0  7  0  0   6  …  10  0  4  6  10  11  0  0  0  0  0
 7  0  9  2  0  0  0  0  0  0  0   2      0  6  0  0   0   0  0  0  0  0  6
 0  0  0  0  0  0  0  0  0  6  0  11      0  0  0  0   0   0  0  0  6  0  0

In [195]:
possort!(jtable)

3×32 Array{Int64,2}:
 0  1  0  2  0  0  0  0  2  0  0  2  …  7  0  7  10  10  11  0  0   0  0  0
 1  0  2  2  0  0  0  0  0  0  0  2     0  7  0   0   0   0  0  0   0  0  9
 0  0  0  0  0  0  0  0  0  1  0  6     0  0  0   0   0   0  0  0  12  0  0

In [196]:
# create initial population of size n
function initpopulation(n,ptable)
    jt = jobtable(ptable)
    popu = zeros(Int,n,size(jt)[1],size(jt)[2])
    for i in 1:n
        p = jobtable(ptable)
        possort!(p)
        popu[i,:,:] = p
    end
    popu
end

initpopulation (generic function with 1 method)

In [204]:
gen1 = initpopulation(100,proctable);

In [205]:
gen1[1,:,:]

3×32 Array{Int64,2}:
 0  1  0  2  0  0  2  0  2  0  0  4  …  0  6  6  7   0  7  0  10  10  0  11
 1  2  0  0  0  2  0  0  0  0  0  0     0  4  0  6   6  0  0   6   7  9   0
 1  0  0  6  0  0  0  0  6  0  0  0     0  0  8  0  11  0  0  12   0  0  12

In [206]:
gen1[2,:,:]

3×32 Array{Int64,2}:
 0  0  0  0  1  2  0  2  0  2  0  0  0  …  0  0  7  0  0  7  10  0  10  0  11
 0  1  2  0  0  0  0  0  0  0  0  2  2     6  0  0  6  0  0   7  0   0  0   9
 0  0  1  0  0  6  0  6  0  0  0  6  8     0  0  0  0  0  0   0  0   0  0   0

## validate job tables

In [173]:
# basically each job number appearing in same column must be exclusive except groupable items until amount of count is below limit.
groupable = [Dict(:id =>[2,6], :cnt => 150)] # annealing can be done simultaneously

1-element Array{Dict{Symbol,Any},1}:
 Dict(:id=>[2, 6],:cnt=>150)

In [177]:
function isgroupable(x, gp)
    for i in 1:length(gp)
        if x in gp[i][:id]
            return(true)
        end
    end
    return(false)
end

isgroupable (generic function with 1 method)

In [180]:
isgroupable(5,groupable)

false

In [181]:
function groupcapacity(x,gp)
    for i in 1:length(gp)
        if x in gp[i][:id]
            return gp[i][:cnt]
        end
    end
    return(0)
end

groupcapacity (generic function with 1 method)

In [183]:
groupcapacity(2,groupable)

150

In [210]:
# check if each column is valid combination of process
function checkvalidity(jtbl,gp,mt)
    flg = true
    k,ll = size(jtbl)
    for i in 1:ll
        q = mt[1,2]
        for j in 1:k
            a = jtbl[j,i]
            for m in (j+1):k
                if jtbl[m,i] == a
                    if isgroupable(a,gp)
                        q += mt[m,2]
                    else
                        flg = false
                    end
                end
            end
            if q > groupcapacity(a,gp)
                flg = false
            end
            if flg == false
                return(flg)
            end
        end
    end
    return(flg)
end

checkvalidity (generic function with 2 methods)

In [215]:
checkvalidity(gen1[5,:,:],groupable,mattable)

false

In [223]:
for i in 1:100
    if !checkvalidity(gen1[i,:,:],groupable,mattable)
        println("id = $i")
        return(gen1[i,:,:])
    end
end

id = 1


3×32 Array{Int64,2}:
 0  1  0  2  0  0  2  0  2  0  0  4  …  0  6  6  7   0  7  0  10  10  0  11
 1  2  0  0  0  2  0  0  0  0  0  0     0  4  0  6   6  0  0   6   7  9   0
 1  0  0  6  0  0  0  0  6  0  0  0     0  0  8  0  11  0  0  12   0  0  12

In [290]:
function validatejob1(jtbl,gp,mt)
    k,ll = size(jtbl)
    for i in 1:ll
        q = mt[1,2]
        for j in 1:k
            a = jtbl[j,i]
            for m in (j+1):k
                if jtbl[m,i] == a
                    if isgroupable(a,gp)
                        q += mt[m,2]
                        if q > groupcapacity(a,gp)
                            jtbl[m,i:end] = circshift(jtbl[m,i:end],1)
                            break
                        end
                    else
                        # circular shift latter data
                        jtbl[m,i:end] = circshift(jtbl[m,i:end],1)
                        break
                    end
                end
            end
        end
    end
    return(possort!(jtbl))
end

validatejob1 (generic function with 1 method)

In [232]:
function validatejob(jtbl,gp,mt,maxcount = 100)
    i = maxcount
    while !checkvalidity(jtbl,gp,mt) | i > 0
        jtbl = validatejob1(jtbl,gp,mt)
        i -= 1
    end
    return(jtbl)
end

validatejob (generic function with 2 methods)

In [291]:
validatejob(gen1[1,:,:],groupable,mattable)

3×32 Array{Int64,2}:
 0  1  0  2  0  0  2  0  2  0  0  4  0  …  6   6   7  0  7  0  10  10   0  11
 1  2  2  0  2  4  0  6  0  6  6  0  7     0   0   0  0  0  0   0   0   0   0
 0  0  0  1  0  6  0  0  6  0  0  0  0     0  11  12  0  0  0   0   0  12   0

## valid length and sorting

In [124]:
# get valid length of job time table
function validlength(jtb)
    l0 = size(jtb)[2]
    l = l0
    for i in l0:-1:1
        if sum(jtb[:,i]) == 0
            l-=1
        else
            break
        end
    end
    return(l)
end

validlength (generic function with 1 method)

In [158]:
function validlengthlist(ptbl)
    ll = size(ptbl)[1]
    tl = zeros(Int, ll)
    for i in 1:ll
        tl[i] = validlength(ptbl[i,:,:])
    end
    tl
end

validlengthlist (generic function with 1 method)

In [235]:
tl = validlengthlist(gen1);

In [236]:
# this is a best proc combination from gen1
minimum(tl)

28

In [237]:
findall(x->x==minimum(tl),tl)

1-element Array{Int64,1}:
 22

In [144]:
# sort population table by its valid length
function sortpopulation(pptbl)
    ll = size(pptbl)[1]
    tl = zeros(Int, ll)
    for i in 1:ll
        tl[i] = validlength(pptbl[i,:,:])
    end
    df = DataFrame(idx = collect(1:ll), tl=tl)
    sort!(df,:tl)
    pptbl[df[:idx],:,:]
end

sortpopulation (generic function with 1 method)

In [238]:
spopu = sortpopulation(gen1);

In [292]:
t1 = spopu[1,:,:]

3×32 Array{Int64,2}:
 0  0  1  2  0  0  0  0  2  2  0  0  0  …   0  10  0   0  10  11  0  0  0  0
 0  0  0  0  0  0  1  2  2  2  0  4  0      7   0  9   0   0   0  0  0  0  0
 1  0  6  0  0  0  0  0  6  0  6  8  0     11  12  0  12   0   0  0  0  0  0

In [295]:
# shrink job table by skipping 0 column
function shrinkjob1!(jtbl)
    flg = false
    cols = validlength(jtbl)
    for j in 1:cols
        if sum(jtbl[:,j]) == 0
            jtbl[:,j:end] = circshift(jtbl[:,j:end],(0,-1))
            flg = true
        end
    end
    return(flg)
end

shrinkjob1! (generic function with 1 method)

In [296]:
function shrinkjob!(jtbl)
    while true
        if !shrinkjob1!(jtbl)
            break
        end
    end
    jtbl
end

shrinkjob! (generic function with 1 method)

In [298]:
shrinkjob!(t1)

3×32 Array{Int64,2}:
 0  1  2  0  0  2  2  0  0  4  6  6  0  …  11  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  2  2  2  0  4  6  0  6  0      0  0  0  0  0  0  0  0  0  0  0
 1  6  0  0  0  6  0  6  8  0  0  0  8      0  0  0  0  0  0  0  0  0  0  0

In [164]:
tl2 = validlengthlist(spopu);

In [169]:
findall(x->x==19,tl2)

2-element Array{Int64,1}:
 1
 2

## survival

In [299]:
# inferior genes cannot live long
function survive(pptbl, rate = 0.8)
    ll = Int(floor(size(pptbl)[1]*rate))
    return(pptbl[1:ll,:,:])
end

survive (generic function with 2 methods)

## cross over and mutation